## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-12-23-50-39 +0000,bbc,Kashmir's young are preserving history - one ...,https://www.bbc.com/news/articles/crr2vq1r7w4o...
1,2025-09-12-23-46-50 +0000,nyt,California Wants to Ban ‘Forever Chemicals’ in...,https://www.nytimes.com/2025/09/12/climate/rac...
2,2025-09-12-23-39-15 +0000,nypost,Florida Dem lawmaker faces demands to resign a...,https://nypost.com/2025/09/12/us-news/dem-spar...
3,2025-09-12-23-39-04 +0000,nypost,Uber sued by federal government over alleged d...,https://nypost.com/2025/09/12/us-news/federal-...
4,2025-09-12-23-35-31 +0000,nypost,Tyler Robinson may have revealed details about...,https://nypost.com/2025/09/12/us-news/tyler-ro...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2369/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,kirk,61
23,charlie,53
373,trump,26
64,shooting,18
65,suspect,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
15,2025-09-12-22-57-31 +0000,wapo,"Charlie Kirk’s widow, Erika Kirk, will deliver...",https://www.washingtonpost.com,220
256,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...,202
14,2025-09-12-22-57-31 +0000,wapo,Live updates: Charlie Kirk shooting suspect in...,https://www.washingtonpost.com/nation/2025/09/...,168
55,2025-09-12-21-12-29 +0000,nyt,Trump Blames the Left and Downplays ‘Radical R...,https://www.nytimes.com/2025/09/12/us/politics...,168
66,2025-09-12-20-56-19 +0000,bbc,"Who is Tyler Robinson, the suspect in custody ...",https://www.bbc.com/news/articles/cp8wl2y66p9o...,166


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
15,220,2025-09-12-22-57-31 +0000,wapo,"Charlie Kirk’s widow, Erika Kirk, will deliver...",https://www.washingtonpost.com
312,56,2025-09-12-01-10-28 +0000,bbc,Man accused of Trump assassination attempt cut...,https://www.bbc.com/news/articles/c8d793z4mzgo...
265,55,2025-09-12-08-38-23 +0000,nypost,Prince Harry makes surprise visit to Kyiv as h...,https://nypost.com/2025/09/12/world-news/princ...
14,54,2025-09-12-22-57-31 +0000,wapo,Live updates: Charlie Kirk shooting suspect in...,https://www.washingtonpost.com/nation/2025/09/...
25,44,2025-09-12-22-26-48 +0000,nypost,NYC synagogue demands proof of voter registrat...,https://nypost.com/2025/09/12/us-news/nyc-syna...
4,41,2025-09-12-23-35-31 +0000,nypost,Tyler Robinson may have revealed details about...,https://nypost.com/2025/09/12/us-news/tyler-ro...
56,39,2025-09-12-21-08-55 +0000,nypost,Ukraine strikes key Russian oil terminal in la...,https://nypost.com/2025/09/12/world-news/ukrai...
202,39,2025-09-12-12-44-13 +0000,bbc,From political outsider to a president like no...,https://www.bbc.com/news/articles/c5yqendv7k5o...
23,37,2025-09-12-22-30-35 +0000,latimes,"California lawmakers pass SB 79, housing bill ...",https://www.latimes.com/california/story/2025-...
41,31,2025-09-12-21-40-00 +0000,wsj,ICE Agent Fatally Shoots Man During Immigratio...,https://www.wsj.com/us-news/man-fatally-shot-i...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
